In [39]:
!pip3 install plotly

In [147]:
def deth_p_slow(steps, time, p_forward=1/3):
    assert p_forward**1 <= 1, 'It must be -1 <= p <= 1'
    if steps < 1:
        return 1
    if time < 1:
        return 0
    return p_forward * deth_p_slow(steps-1, time-1, p_forward=p_forward) + \
           (1 - p_forward) * deth_p_slow(steps+1, time-1, p_forward=p_forward)

In [148]:
deth_p_slow(1, 20)

0.49168868385103226

In [151]:
def deth_p_fast(steps, time, p_forward=1, cash={}):
    assert p_forward >= 0 and p_forward <= 1, 'It must be 0 <= p <= 1'
    if (steps, time, p_forward) in cash.keys():
        return cash[(steps, time, p_forward)]
    if steps < 1:
        return 1
    if time < 1:
        return 0
    cash[(steps, time, p_forward)] = p_forward * deth_p_fast(steps-1, time-1, p_forward=p_forward) + \
                          (1 - p_forward) * deth_p_fast(steps+1, time-1, p_forward=p_forward)
    return cash[(steps, time, p_forward)]

In [152]:
deth_p_fast(2, 2,  p_forward=0.5)

0.25

In [261]:
def alive_p(steps, time, p_forward=1/3, func=deth_p_fast):
    return 1-func(steps, time, p_forward=p_forward)

In [262]:
alive_p(2,2, p_forward=0.5)

0.75

In [275]:
import sys
import plotly.offline as py
from plotly.graph_objs import *
py.init_notebook_mode(connected=True)

def x_steps_close(steps=5, time_range=20, p_forward=[0.333]):
    sys.setrecursionlimit(time_range**2 if time_range > 1000 else 1100)
    probs = [alive_p(5, i) for i in range(time_range)]
    traces = [Scatter(x=list(range(time_range)), \
                      y=[alive_p(steps, i, p_forward=p, func=deth_p_fast) for i in range(time_range)], \
                      name=p) \
              for p in p_forward]
    py.iplot(traces, filename='trace.html')

In [278]:
x_steps_close(steps=5, time_range=2000, p_forward=[0.5, 0.49])